# Spatial Autocorrelation

This notebook benchmarks the spatial autocorrelation (Moran's I and Geary's C) for `squidpy` and `rapids-singlecell`.

In [1]:
import rapids_singlecell as rsc
import squidpy as sq
import cupy as cp

In [2]:
import rmm
from rmm.allocators.cupy import rmm_cupy_allocator

rmm.reinitialize(
    managed_memory=False,  # Allows oversubscription
    pool_allocator=False,  # default is False
    devices=0,  # GPU device IDs to register. By default registers only GPU 0.
)
cp.cuda.set_allocator(rmm_cupy_allocator)

In [3]:
adata = sq.datasets.visium_hne_adata()

We first need to compute a spatial graph with `squidpy.gr.spatial_neighbors()`.

In [4]:
%%time
sq.gr.spatial_neighbors(adata)

CPU times: user 21.8 ms, sys: 642 µs, total: 22.5 ms
Wall time: 21.7 ms


In [5]:
genes = adata.var_names

## Moran's I

### GPU

In [6]:
%%timeit
rsc.gr.spatial_autocorr(adata, mode="moran", genes=genes, n_perms=100, use_sparse=False)

420 ms ± 21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
adata.uns["moranI"]

,I,pval_norm,var_norm,pval_z_sim,pval_sim,var_sim,pval_norm_fdr_bh,pval_z_sim_fdr_bh,pval_sim_fdr_bh
Nrgn,0.874753,0.000000,0.000131,0.000000,0.009901,0.000364,0.000000,0.000000,0.016677
Mbp,0.868723,0.000000,0.000131,0.000000,0.009901,0.000328,0.000000,0.000000,0.016677
Camk2n1,0.866545,0.000000,0.000131,0.000000,0.009901,0.000334,0.000000,0.000000,0.016677
Slc17a7,0.861761,0.000000,0.000131,0.000000,0.009901,0.000337,0.000000,0.000000,0.016677
Ttr,0.841988,0.000000,0.000131,0.000000,0.009901,0.000314,0.000000,0.000000,0.016677
...,...,...,...,...,...,...,...,...,...
Slc9a8,-0.026960,0.010027,0.000131,0.000190,0.009901,0.000060,0.017979,0.000362,0.016677
Pramef8,-0.028659,0.006681,0.000131,0.000217,0.009901,0.000064,0.012348,0.000411,0.016677
Klf12,-0.028764,0.006512,0.000131,0.000103,0.009901,0.000055,0.012052,0.000201,0.016677
Gart,-0.028859,0.006361,0.000131,0.000192,0.009901,0.000068,0.011784,0.000366,0.016677


### CPU

In [8]:
%%time
sq.gr.spatial_autocorr(adata, mode="moran", genes=genes, n_perms=100, n_jobs=32)

  0%|          | 0/100 [00:00<?, ?/s]

CPU times: user 4min 53s, sys: 2.32 s, total: 4min 55s
Wall time: 41.3 s


In [9]:
adata.uns["moranI"]

,I,pval_norm,var_norm,pval_z_sim,pval_sim,var_sim,pval_norm_fdr_bh,pval_z_sim_fdr_bh,pval_sim_fdr_bh
Nrgn,0.874753,0.000000,0.000131,0.000000,0.009901,0.000300,0.000000,0.000000,0.016675
Mbp,0.868723,0.000000,0.000131,0.000000,0.009901,0.000289,0.000000,0.000000,0.016675
Camk2n1,0.866542,0.000000,0.000131,0.000000,0.009901,0.000376,0.000000,0.000000,0.016675
Slc17a7,0.861761,0.000000,0.000131,0.000000,0.009901,0.000295,0.000000,0.000000,0.016675
Ttr,0.841986,0.000000,0.000131,0.000000,0.009901,0.000302,0.000000,0.000000,0.016675
...,...,...,...,...,...,...,...,...,...
Slc9a8,-0.026960,0.010026,0.000131,0.000061,0.009901,0.000052,0.017977,0.000121,0.016675
Pramef8,-0.028656,0.006687,0.000131,0.000135,0.009901,0.000057,0.012356,0.000261,0.016675
Klf12,-0.028760,0.006518,0.000131,0.000090,0.009901,0.000057,0.012061,0.000176,0.016675
Gart,-0.028857,0.006364,0.000131,0.000130,0.009901,0.000063,0.011789,0.000251,0.016675


## Geary's C

### GPU

In [10]:
%%timeit
rsc.gr.spatial_autocorr(adata, mode="geary", genes=genes, n_perms=100, use_sparse=False)

411 ms ± 512 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
adata.uns["gearyC"]

,C,pval_norm,var_norm,pval_z_sim,pval_sim,var_sim,pval_norm_fdr_bh,pval_z_sim_fdr_bh,pval_sim_fdr_bh
Nrgn,0.126481,0.000000e+00,0.000131,0.000000,0.009901,0.000393,0.000000e+00,0.000000,0.016691
Mbp,0.132854,0.000000e+00,0.000131,0.000000,0.009901,0.000341,0.000000e+00,0.000000,0.016691
Camk2n1,0.135106,0.000000e+00,0.000131,0.000000,0.009901,0.000347,0.000000e+00,0.000000,0.016691
Slc17a7,0.138819,0.000000e+00,0.000131,0.000000,0.009901,0.000399,0.000000e+00,0.000000,0.016691
Ttr,0.156155,0.000000e+00,0.000131,0.000000,0.009901,0.000250,0.000000e+00,0.000000,0.016691
...,...,...,...,...,...,...,...,...,...
AC166344.1,1.079271,2.059686e-12,0.000131,0.403457,0.099010,0.000047,7.064124e-12,0.425065,0.133336
Gm14091,1.083541,1.372236e-13,0.000131,0.380466,0.188119,0.000083,4.932845e-13,0.411113,0.231301
Slc6a5,1.087126,1.265654e-14,0.000131,0.368844,0.376238,0.000058,4.722497e-14,0.403410,0.404281
Soat2,1.093646,1.110223e-16,0.000131,0.438600,0.297030,0.000129,4.432556e-16,0.449287,0.335355


### CPU

In [12]:
%%time
sq.gr.spatial_autocorr(adata, mode="geary", genes=genes, n_perms=100, n_jobs=32)

  0%|          | 0/100 [00:00<?, ?/s]

CPU times: user 5min, sys: 273 ms, total: 5min
Wall time: 37.5 s


In [13]:
adata.uns["gearyC"]

,C,pval_norm,var_norm,pval_z_sim,pval_sim,var_sim,pval_norm_fdr_bh,pval_z_sim_fdr_bh,pval_sim_fdr_bh
Nrgn,0.126481,0.000000e+00,0.000131,0.000000,0.009901,0.000282,0.000000e+00,0.000000,0.016764
Mbp,0.132855,0.000000e+00,0.000131,0.000000,0.009901,0.000315,0.000000e+00,0.000000,0.016764
Camk2n1,0.135106,0.000000e+00,0.000131,0.000000,0.009901,0.000269,0.000000e+00,0.000000,0.016764
Slc17a7,0.138820,0.000000e+00,0.000131,0.000000,0.009901,0.000317,0.000000e+00,0.000000,0.016764
Ttr,0.156156,0.000000e+00,0.000131,0.000000,0.009901,0.000312,0.000000e+00,0.000000,0.016764
...,...,...,...,...,...,...,...,...,...
AC166344.1,1.079269,2.062794e-12,0.000131,0.432510,0.504950,0.000027,7.074786e-12,0.443551,0.504950
Gm14091,1.083544,1.368905e-13,0.000131,0.408545,0.475248,0.000173,4.920872e-13,0.427605,0.492916
Slc6a5,1.087121,1.276756e-14,0.000131,0.379592,0.495050,0.000051,4.762939e-14,0.409957,0.498969
Soat2,1.093644,1.110223e-16,0.000131,0.425606,0.475248,0.000307,4.432626e-16,0.438888,0.492916
